# Automatic atom mapping - example

This notebook demonstrates how the `atom_mapping` module works in practice. It's purpose is to reduce the workload when preparing input data for MFA analysis on INCA. 

The only input required is the COBRA model that contains all reaction data, and, most importantly, references for metabolite structures in KEGG Compound, HMDB, CHEBI databases, or an InChI key.

This module uses RDKit to generate the input for RDT (Reaction Decoder Tool), a tool to estimate the atom mapping of a reaction. The output of RDT is then parsed and used to generate a dictionary of atom mappings for each reaction in the model.

#### First, import required modules:

In [2]:
import pandas as pd
from incawrapper.atommapping import atom_mapping
import re

## Overview

#### 1. create a dataframe with the reactions
- df that has rxn_id and annotations as columns
- the information is taken from the model

In [5]:
file = "./Literature data/wasylenko/wasylenko_model_KEGG.csv"

wasylenko_kegg = pd.read_csv(file, index_col=0)
wasylenko_kegg

,0
0,C00031.ext (abcdef) -> C00092 (abcdef)
1,C01510.ext (abcde) -> C00231 (abcde)
2,C00469 (ab) -> C00469.ext (ab)
3,C00033 (ab) -> C00033.ext (ab)
4,C00116 (abc) -> C00116.ext (abc)
...,...
90,C00092 (abcdef) -> C00182 (abcdef)
91,C00092 (abcdef) -> C00965 (abcdef)
92,C05345 (abcdef) -> C00464 (abcdef)
93,C00011.ext (a) -> C00011 (a)


In [6]:
rxn = wasylenko_kegg.iloc[5, 0]
rxn

'0.459 C00041 + 0.161 C00062 + 0.102 C00152 + 0.297 C00049 + 0.007 C00097 + 0.105 C00064 + 0.302 C00025 + 0.29 C00037 + 0.066 C00135 + 0.193 C00407 + 0.296 C00123 + 0.286 C00047 + 0.051 C00073 + 0.134 C00079 + 0.165 C00148 + 0.185 C00065 + 0.191 C00188 + 0.028 C00078 + 0.102 C00082 + 0.265 C00183 + 0.519 C00182 + 0.023 Trehalose + 0.808 C00464 + 1.135 C00965 + 0.051 C00020 + 0.05 C00055 + 0.051 C00144 + 0.067 C00105 + 0.0036 C00360 + 0.0024 C00239 + 0.0024 C00362 + 0.0036 C00364 + 0.0066 C00422 + 0.0007 C01694 + 0.0015 C05437 + 0.0006 PA + 0.0062 PC + 0.0046 PE + 0.0055 PI + 0.0017 PS + 0.4623 C00024.c -> Biomass'

Remove atom mapping

In [7]:
def remove_atommapping(rxn):
    rxn_wo_am = re.sub(r'\([^)]*\)', '', rxn)
    return rxn_wo_am

rxn_wo_am = remove_atommapping(rxn)
rxn_wo_am

'0.459 C00041 + 0.161 C00062 + 0.102 C00152 + 0.297 C00049 + 0.007 C00097 + 0.105 C00064 + 0.302 C00025 + 0.29 C00037 + 0.066 C00135 + 0.193 C00407 + 0.296 C00123 + 0.286 C00047 + 0.051 C00073 + 0.134 C00079 + 0.165 C00148 + 0.185 C00065 + 0.191 C00188 + 0.028 C00078 + 0.102 C00082 + 0.265 C00183 + 0.519 C00182 + 0.023 Trehalose + 0.808 C00464 + 1.135 C00965 + 0.051 C00020 + 0.05 C00055 + 0.051 C00144 + 0.067 C00105 + 0.0036 C00360 + 0.0024 C00239 + 0.0024 C00362 + 0.0036 C00364 + 0.0066 C00422 + 0.0007 C01694 + 0.0015 C05437 + 0.0006 PA + 0.0062 PC + 0.0046 PE + 0.0055 PI + 0.0017 PS + 0.4623 C00024.c -> Biomass'

Split substrate and product side

In [8]:
def split_rxn(rxn_wo_am):
    if " <-> " in rxn_wo_am:
        rxn_sub, rxn_prod = rxn_wo_am.split(" <-> ")
    elif " -> " in rxn_wo_am:
        rxn_sub, rxn_prod = rxn_wo_am.split(" -> ")
    return rxn_sub, rxn_prod

rxn_sub, rxn_prod = split_rxn(rxn_wo_am)

print(rxn_sub, rxn_prod)

0.459 C00041 + 0.161 C00062 + 0.102 C00152 + 0.297 C00049 + 0.007 C00097 + 0.105 C00064 + 0.302 C00025 + 0.29 C00037 + 0.066 C00135 + 0.193 C00407 + 0.296 C00123 + 0.286 C00047 + 0.051 C00073 + 0.134 C00079 + 0.165 C00148 + 0.185 C00065 + 0.191 C00188 + 0.028 C00078 + 0.102 C00082 + 0.265 C00183 + 0.519 C00182 + 0.023 Trehalose + 0.808 C00464 + 1.135 C00965 + 0.051 C00020 + 0.05 C00055 + 0.051 C00144 + 0.067 C00105 + 0.0036 C00360 + 0.0024 C00239 + 0.0024 C00362 + 0.0036 C00364 + 0.0066 C00422 + 0.0007 C01694 + 0.0015 C05437 + 0.0006 PA + 0.0062 PC + 0.0046 PE + 0.0055 PI + 0.0017 PS + 0.4623 C00024.c Biomass


Split reactants

In [9]:
def split_reactants_products(rxn_side):
    if " + " in rxn_side:
        rxn_side_split = rxn_side.split(" + ")
    else:
        rxn_side_split = [rxn_side]
    return rxn_side_split

rxn_sub_split = split_reactants_products(rxn_sub)
rxn_prod_split = split_reactants_products(rxn_prod)

print(rxn_sub_split, rxn_prod_split)

['0.459 C00041', '0.161 C00062', '0.102 C00152', '0.297 C00049', '0.007 C00097', '0.105 C00064', '0.302 C00025', '0.29 C00037', '0.066 C00135', '0.193 C00407', '0.296 C00123', '0.286 C00047', '0.051 C00073', '0.134 C00079', '0.165 C00148', '0.185 C00065', '0.191 C00188', '0.028 C00078', '0.102 C00082', '0.265 C00183', '0.519 C00182', '0.023 Trehalose', '0.808 C00464', '1.135 C00965', '0.051 C00020', '0.05 C00055', '0.051 C00144', '0.067 C00105', '0.0036 C00360', '0.0024 C00239', '0.0024 C00362', '0.0036 C00364', '0.0066 C00422', '0.0007 C01694', '0.0015 C05437', '0.0006 PA', '0.0062 PC', '0.0046 PE', '0.0055 PI', '0.0017 PS', '0.4623 C00024.c'] ['Biomass']


In [10]:
def clear_white_spaces(compound_list: list):
    new_list = []
    for compound in compound_list:
        new_compound = re.sub(r'[ ]+', ' ', compound)
        new_compound = new_compound.strip()
        new_list.append(new_compound)
    return new_list

rxn_sub_split = clear_white_spaces(rxn_sub_split)
rxn_prod_split = clear_white_spaces(rxn_prod_split)

print(rxn_sub_split, rxn_prod_split)

['0.459 C00041', '0.161 C00062', '0.102 C00152', '0.297 C00049', '0.007 C00097', '0.105 C00064', '0.302 C00025', '0.29 C00037', '0.066 C00135', '0.193 C00407', '0.296 C00123', '0.286 C00047', '0.051 C00073', '0.134 C00079', '0.165 C00148', '0.185 C00065', '0.191 C00188', '0.028 C00078', '0.102 C00082', '0.265 C00183', '0.519 C00182', '0.023 Trehalose', '0.808 C00464', '1.135 C00965', '0.051 C00020', '0.05 C00055', '0.051 C00144', '0.067 C00105', '0.0036 C00360', '0.0024 C00239', '0.0024 C00362', '0.0036 C00364', '0.0066 C00422', '0.0007 C01694', '0.0015 C05437', '0.0006 PA', '0.0062 PC', '0.0046 PE', '0.0055 PI', '0.0017 PS', '0.4623 C00024.c'] ['Biomass']


Extract stoichiometry

In [11]:
def extract_stoichiometry_and_compound(rxn_side_split):
    rxn_side_stoich = []
    rxn_side_compound = []
    for reactant in rxn_side_split:
        if " " in reactant:
            stoich, compound = reactant.split(" ")
            rxn_side_stoich.append(stoich)
            #if "." in compound:
            #    compound = compound.split(".")[0]
            rxn_side_compound.append(compound)
        else:
            rxn_side_stoich.append("1")
            #if "." in reactant:
            #    reactant = reactant.split(".")[0]
            rxn_side_compound.append(reactant)
    return rxn_side_stoich, rxn_side_compound

rxn_sub_stoich, rxn_sub_compound = extract_stoichiometry_and_compound(rxn_sub_split)
rxn_prod_stoich, rxn_prod_compound = extract_stoichiometry_and_compound(rxn_prod_split)

print(rxn_sub_stoich, rxn_sub_compound)
print(rxn_prod_stoich, rxn_prod_compound)

['0.459', '0.161', '0.102', '0.297', '0.007', '0.105', '0.302', '0.29', '0.066', '0.193', '0.296', '0.286', '0.051', '0.134', '0.165', '0.185', '0.191', '0.028', '0.102', '0.265', '0.519', '0.023', '0.808', '1.135', '0.051', '0.05', '0.051', '0.067', '0.0036', '0.0024', '0.0024', '0.0036', '0.0066', '0.0007', '0.0015', '0.0006', '0.0062', '0.0046', '0.0055', '0.0017', '0.4623'] ['C00041', 'C00062', 'C00152', 'C00049', 'C00097', 'C00064', 'C00025', 'C00037', 'C00135', 'C00407', 'C00123', 'C00047', 'C00073', 'C00079', 'C00148', 'C00065', 'C00188', 'C00078', 'C00082', 'C00183', 'C00182', 'Trehalose', 'C00464', 'C00965', 'C00020', 'C00055', 'C00144', 'C00105', 'C00360', 'C00239', 'C00362', 'C00364', 'C00422', 'C01694', 'C05437', 'PA', 'PC', 'PE', 'PI', 'PS', 'C00024.c']
['1'] ['Biomass']


Run on all reactions

In [12]:
model_dict = {}

for i, row in wasylenko_kegg.iterrows():
    rxn = row[0]
    rxn_wo_am = remove_atommapping(rxn)
    rxn_sub, rxn_prod = split_rxn(rxn_wo_am)
    rxn_sub_split = split_reactants_products(rxn_sub)
    rxn_prod_split = split_reactants_products(rxn_prod)
    rxn_sub_split_cleared = clear_white_spaces(rxn_sub_split)
    rxn_prod_split_cleared = clear_white_spaces(rxn_prod_split)
    rxn_sub_stoich, rxn_sub_compound = extract_stoichiometry_and_compound(rxn_sub_split_cleared)
    rxn_prod_stoich, rxn_prod_compound = extract_stoichiometry_and_compound(rxn_prod_split_cleared)
    model_dict[i] = {
        "rxn_id": i,
        "equation": rxn_wo_am,
        "reactants_stoichiometry": [float(stoich) * -1 for stoich in rxn_sub_stoich],
        "reactants_ids": rxn_sub_compound,
        "products_stoichiometry": [float(stoich) for stoich in rxn_prod_stoich],
        "products_ids": rxn_prod_compound,
    }

rxn_data = pd.DataFrame.from_dict(model_dict, orient='index')
rxn_data.head(10)

,rxn_id,equation,reactants_stoichiometry,reactants_ids,products_stoichiometry,products_ids
0,0,C00031.ext -> C00092,[-1.0],[C00031.ext],[1.0],[C00092]
1,1,C01510.ext -> C00231,[-1.0],[C01510.ext],[1.0],[C00231]
2,2,C00469 -> C00469.ext,[-1.0],[C00469],[1.0],[C00469.ext]
3,3,C00033 -> C00033.ext,[-1.0],[C00033],[1.0],[C00033.ext]
4,4,C00116 -> C00116.ext,[-1.0],[C00116],[1.0],[C00116.ext]
5,5,0.459 C00041 + 0.161 C00062 + 0.102 C00152 + 0...,"[-0.459, -0.161, -0.102, -0.297, -0.007, -0.10...","[C00041, C00062, C00152, C00049, C00097, C0006...",[1.0],[Biomass]
6,6,C00022.c -> C00022.mnt,[-1.0],[C00022.c],[1.0],[C00022.mnt]
7,7,C00022.m -> C00022.mnt,[-1.0],[C00022.m],[1.0],[C00022.mnt]
8,8,C00022.mnt -> C00022.fix,[-1.0],[C00022.mnt],[1.0],[C00022.fix]
9,9,C00092 <-> C05345,[-1.0],[C00092],[1.0],[C05345]


#### 2. create a dataframe with the metabolites
- df that has met_id and annoations as columns

In [14]:
metabolites = []
for i, row in rxn_data.iterrows():
    metabolites.extend(row["reactants_ids"])
    metabolites.extend(row["products_ids"])
metabolites = list(set(metabolites))
metabolites

met_df = {}
for met in metabolites:
    if met.startswith("C"):
        if "." in met:
            temp_met = met.split(".")[0]
        else:
            temp_met = met
        met_df[met] = {
            "met_id": met,
            "annotations": {"kegg.compound": temp_met},
        }

met_df = pd.DataFrame.from_dict(met_df, orient='index')
met_df

,met_id,annotations
C00111,C00111,{'kegg.compound': 'C00111'}
C00037,C00037,{'kegg.compound': 'C00037'}
C00073,C00073,{'kegg.compound': 'C00073'}
C00011.out,C00011.out,{'kegg.compound': 'C00011'}
C00122,C00122,{'kegg.compound': 'C00122'}
...,...,...
C00279,C00279,{'kegg.compound': 'C00279'}
C00078,C00078,{'kegg.compound': 'C00078'}
C00451,C00451,{'kegg.compound': 'C00451'}
C00356,C00356,{'kegg.compound': 'C00356'}


#### 3. create a MolfileDownloader object
- calling the generate_molfile_database method triggers the download of the molfiles for the metabolites in the dataframe

In [15]:
base_path = "/Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko"
downloader = atom_mapping.MolfileDownloader(met_df, base_path=base_path)
downloader.generate_molfile_database()

Fetching metabolite structures...


Successfully fetched 79/81 metabolites


#### 4. Write reactions in the correct format for RDT (write_rxn_file)

In [16]:
met_df.query("met_id.str.contains('C00469')")

,met_id,annotations
C00469.ext,C00469.ext,{'kegg.compound': 'C00469'}
C00469,C00469,{'kegg.compound': 'C00469'}


In [17]:
atom_mapping.write_rxn_files(rxn_data, base_path=base_path)

Metabolite structures missing for reaction 5
Metabolite structures missing for reaction 21
Metabolite structures missing for reaction 22
Metabolite structures missing for reaction 23
Metabolite structures missing for reaction 24
Metabolite structures missing for reaction 25
Metabolite structures missing for reaction 41
Metabolite structures missing for reaction 42
Metabolite structures missing for reaction 57
Metabolite structures missing for reaction 65
Metabolite structures missing for reaction 68
Metabolite structures missing for reaction 75
Metabolite structures missing for reaction 76
Metabolite structures missing for reaction 77
Metabolite structures missing for reaction 78
Metabolite structures missing for reaction 79
Metabolite structures missing for reaction 80
Metabolite structures missing for reaction 88
Metabolite structures missing for reaction 89
Metabolite structures missing for reaction 92
Generated 75/95


#### 5. Query RDT for the atom mapping (obtain_atom_mappings)
- This might take a while
- Can be skipped if you already have the atom mappings

In [18]:
atom_mapping.obtain_atom_mappings(max_time=20, base_path=base_path)

openjdk version "11.0.9.1" 2020-11-04 LTS
OpenJDK Runtime Environment Zulu11.43+55-CA (build 11.0.9.1+1-LTS)
OpenJDK 64-Bit Server VM Zulu11.43+55-CA (build 11.0.9.1+1-LTS, mixed mode)


Mapping reactions...
['81.rxn', '4.rxn', '56.rxn', '43.rxn', '94.rxn', '82.rxn', '55.rxn', '7.rxn', '69.rxn', '40.rxn', '54.rxn', '6.rxn', '83.rxn', '93.rxn', '87.rxn', '2.rxn', '50.rxn', '44.rxn', '45.rxn', '3.rxn', '51.rxn', '86.rxn', '84.rxn', '90.rxn', '47.rxn', '53.rxn', '1.rxn', '52.rxn', '0.rxn', '46.rxn', '91.rxn', '85.rxn', '35.rxn', '34.rxn', '20.rxn', '36.rxn', '37.rxn', '33.rxn', '27.rxn', '26.rxn', '32.rxn', '18.rxn', '30.rxn', '31.rxn', '19.rxn', '14.rxn', '28.rxn', '29.rxn', '15.rxn', '17.rxn', '16.rxn', '12.rxn', '13.rxn', '39.rxn', '11.rxn', '10.rxn', '38.rxn', '63.rxn', '62.rxn', '74.rxn', '60.rxn', '48.rxn', '49.rxn', '61.rxn', '59.rxn', '71.rxn', '64.rxn', '70.rxn', '58.rxn', '8.rxn', '66.rxn', '72.rxn', '73.rxn', '67.rxn', '9.rxn']
!--------------------------------------------------------
Reaction Decoder Tool (RDT)

Author: Syed Asad Rahman
e-mail: asad@ebi.ac.uk|s9asad@gmail.com
c/o EMBL-European BioInformatics Institute (EBI)
WTGC, CB10 1SD Hinxton
UK

Note: The

uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: Number of atom(s) on the Left side 10 =/= Number of atom(s) on the Right side 15
uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: {C=3, O=6, P=1} =/= {C=6, O=6, R=3}
uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: Unmapped atoms present in this reaction(MIN) algorithm.


0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
8    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature%20data/wasylenko/RDT.jar!/META-INF/jnati/jnati.default-properties
10   [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
11   [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading instance defaults: jar:file:/Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature%20data/wasylenko/RDT.jar!/META-INF/jnati/jnati.instance.default-properties
13   [main] INFO  net.sf.jnati.deploy.repository.ClasspathRepository  - Searching classpath for: jniinchi-1.03_1-MAC-X86_64
14   [main] INFO  net.sf.jnati.deploy.repository.LocalRepository  - Searching local repository for: jniinchi-1.03_1-MAC-X86_64
14   [main] DEBUG net.sf.jnati.deploy.repository.LocalRepository  - Artefact

uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: Unmapped atoms present in this reaction(MAX) algorithm.
uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: Unmapped atoms present in this reaction(MIXTURE) algorithm.


...

uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: Unmapped atoms present in this reaction(MAX) algorithm.
uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: Unmapped atoms present in this reaction(MIXTURE) algorithm.


Mapped RXN File /Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko/mappedRxns/ECBLAST_73_AAM.rxn


Annotated RXN Image /Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko/mappedRxns/ECBLAST_73_AAM.png
Output is presented in text format: /Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko/mappedRxns/ECBLAST_73_AAM.txt
!--------------------------------------------------------
Reaction Decoder Tool (RDT)

Author: Syed Asad Rahman
e-mail: asad@ebi.ac.uk|s9asad@gmail.com
c/o EMBL-European BioInformatics Institute (EBI)
WTGC, CB10 1SD Hinxton
UK

Note: The copyright of this software belongs to the author
and EMBL-European BioInformatics Institute (EBI).

Reference
Rahman, S.A. et.al.(2016) Reaction Decoder Tool (RDT):
Extracting Features from Chemical Reactions, Bioinformatics (2016)
doi: 10.1093/bioinformatics/btw096
!--------------------------------------------------------

-- AAM --
0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
9    [main] DEBUG net.sf.jnati.deploy.artefact.Conf

Annotated RXN Image /Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko/mappedRxns/ECBLAST_67_AAM.png
Output is presented in text format: /Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko/mappedRxns/ECBLAST_67_AAM.txt
!--------------------------------------------------------
Reaction Decoder Tool (RDT)

Author: Syed Asad Rahman
e-mail: asad@ebi.ac.uk|s9asad@gmail.com
c/o EMBL-European BioInformatics Institute (EBI)
WTGC, CB10 1SD Hinxton
UK

Note: The copyright of this software belongs to the author
and EMBL-European BioInformatics Institute (EBI).

Reference
Rahman, S.A. et.al.(2016) Reaction Decoder Tool (RDT):
Extracting Features from Chemical Reactions, Bioinformatics (2016)
doi: 10.1093/bioinformatics/btw096
!--------------------------------------------------------

-- AAM --
0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
7    [main] DEBUG net.sf.jnati.deploy.artefact.Conf

Annotated RXN Image /Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko/mappedRxns/ECBLAST_9_AAM.png
Output is presented in text format: /Users/krv114/Documents/GitHub/incawrapper/docs/examples/Literature data/wasylenko/mappedRxns/ECBLAST_9_AAM.txt
Reactions mapped in total: 75/75


#### 6. Write reactions file with atom mapping

In [19]:
atom_mapping.parse_single_reaction_mappings(base_path + '/mappedRxns/rxnFiles/48.rxn', rxn_data.iloc[48]['equation'])

'1.0*C00025 (C1:a C2:b C3:c C4:d C5:e) -> 1.0*C00148 (C1:c C2:a C3:b C4:d C5:e)'

In [20]:
atom_mapping.parse_reaction_mappings(mapped_rxn_path=base_path + '/mappedRxns/rxnFiles', rxn_data=rxn_data)

5 not mapped
21 not mapped
22 not mapped
23 not mapped
24 not mapped
25 not mapped
 has unmapped carbon(-s)
 has unmapped carbon(-s)
41 not mapped
42 not mapped
57 not mapped
65 not mapped
68 not mapped
75 not mapped
76 not mapped
77 not mapped
78 not mapped
79 not mapped
80 not mapped
 has unmapped carbon(-s)
 has unmapped carbon(-s)
88 not mapped
89 not mapped
 has unmapped carbon(-s)
92 not mapped


,rxn_id,equation
0,0,1.0*C00031.ext (C1:a C2:b C3:c C4:d C5:e C6:f)...
1,1,1.0*C01510.ext (C1:a C2:b C3:c C4:d C5:e) -> 1...
2,2,1.0*C00469 (C1:a C2:b) -> 1.0*C00469.ext (C1:a...
3,3,1.0*C00033 (C1:a C2:b) -> 1.0*C00033.ext (C1:a...
4,4,1.0*C00116 (C1:a C2:b C3:c) -> 1.0*C00116.ext ...
...,...,...
65,85,1.0*C00418 (C1:a C2:b C3:c C4:d C5:e C6:f) -> ...
66,86,1.0*C00129 (C1:a C2:b C3:c C4:d C5:e) -> 1.0*C...
67,91,1.0*C00092 (C1:a C2:b C3:c C4:d C5:e C6:f) -> ...
68,93,1.0*C00011.ext (C1:a) -> 1.0*C00011 (C1:a)


In [21]:
wasylenko_kegg

,0
0,C00031.ext (abcdef) -> C00092 (abcdef)
1,C01510.ext (abcde) -> C00231 (abcde)
2,C00469 (ab) -> C00469.ext (ab)
3,C00033 (ab) -> C00033.ext (ab)
4,C00116 (abc) -> C00116.ext (abc)
...,...
90,C00092 (abcdef) -> C00182 (abcdef)
91,C00092 (abcdef) -> C00965 (abcdef)
92,C05345 (abcdef) -> C00464 (abcdef)
93,C00011.ext (a) -> C00011 (a)


#### 7. When you're done, you can delete the downloaded files and directories to free up space

In [18]:
atom_mapping.clean_output(base_path=base_path)